In [1]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [2]:
full_table = pd.read_csv('./data/processed_results/osc_df')

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_231"; Java(TM) SE Runtime Environment (build 1.8.0_231-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from /Users/nickparker/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmp4is6xjx0
  JVM stdout: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmp4is6xjx0/h2o_nickparker_started_from_python.out
  JVM stderr: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmp4is6xjx0/h2o_nickparker_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_nickparker_dng0zp
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


### Train/Test Split

First Year of Existance:
- gg_drama 1943
- gg_comedy 1951
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [4]:
# I pick a min_year where the awards shows will be relevant
min_year = 1995

# Leftover code from designing the split
# # 80/20 Split (random)
# table_sized = full_table.loc[(full_table['year'] >= min_year)]
# rand_rows = np.random.choice(table_sized.shape[0], table_sized.shape[0])
# split = int(0.8 * len(rand_rows))
# train = table_sized.iloc[rand_rows[:split]]
# valid = table_sized.iloc[rand_rows[split:]]

# # 80/20 Split (sequential)
# year_range = 2019 - min_year
# train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
# valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

# # Final Prediction
# train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] >= min_year))]
# test = full_table.loc[(full_table['year'] == 2019)]

# train.to_csv('./data/processed_results/train.csv', index = False)
# valid.to_csv('./data/processed_results/valid.csv', index = False)

# H2O's Auto ML

## Pick best model by validating with the first 80% of the data

In [5]:
# 80/20 Split (sequential) Excluding 2019
year_range = 2019 - min_year
train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

In [6]:
print('training set contains:', train.shape[0], 'movies')
print('validation set contains:', valid.shape[0], 'movies')

training set contains: 125 movies
validation set contains: 34 movies


In [7]:
train.columns

Index(['Unnamed: 0', 'year', 'film', 'wiki', 'winner', 'Nominations',
       'Oscar_win', 'nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy',
       'winner_gg_comedy', 'nom_pga', 'winner_pga', 'nom_bafta',
       'winner_bafta', 'nom_dga', 'winner_dga', 'nom_sag', 'winner_sag',
       'nom_cannes', 'winner_cannes'],
      dtype='object')

In [8]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(valid)

# Identify predictors and response
predictors = ['year','nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
        'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes','Nominations']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |███████████
10:11:13.879: Skipping training of model GBM_5_AutoML_20200125_101057 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200125_101057.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 125.0.


█████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
XGBoost_grid__1_AutoML_20200125_101057_model_4,0.899048,0.344447,0.462829,0.208333,0.338942,0.114882,211,0.024542
DeepLearning_1_AutoML_20200125_101057,0.88619,0.430868,0.421092,0.125,0.348038,0.121131,32,0.017899
XGBoost_grid__1_AutoML_20200125_101057_model_1,0.884762,0.481381,0.413137,0.125,0.392884,0.154358,225,0.021286
StackedEnsemble_AllModels_AutoML_20200125_101057,0.882857,0.337347,0.397983,0.130952,0.328002,0.107585,167,0.342286
StackedEnsemble_BestOfFamily_AutoML_20200125_101057,0.881429,0.333825,0.394172,0.121429,0.340589,0.116001,234,0.078971
XRT_1_AutoML_20200125_101057,0.811905,0.439931,0.35452,0.22381,0.36671,0.134476,76,0.026546
DRF_1_AutoML_20200125_101057,0.785714,0.731257,0.307786,0.202381,0.381997,0.145921,95,0.027219
XGBoost_grid__1_AutoML_20200125_101057_model_2,0.748333,0.416425,0.240409,0.147619,0.364964,0.133198,142,0.022895
GLM_1_AutoML_20200125_101057,0.704286,0.432762,0.32774,0.261905,0.354253,0.125495,32,0.016544
GBM_3_AutoML_20200125_101057,0.681905,0.560564,0.287336,0.261905,0.376151,0.14149,58,0.028393


## Once best model is picked, retrain that on the entire dataset

In [9]:
top_model = aml.leader
top_model

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid__1_AutoML_20200125_101057_model_4


Model Summary: 


,,number_of_trees
0,,134.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.04634662654427943
RMSE: 0.21528266661363946
LogLoss: 0.17710936267777422
Mean Per-Class Error: 0.0535714285714286
AUC: 0.9809523809523809
AUCPR: 0.8604749365365195
Gini: 0.9619047619047618

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.47652560472488403: 


,,0,1,Error,Rate
0,0,102.0,3.0,0.0286,(3.0/105.0)
1,1,3.0,17.0,0.15,(3.0/20.0)
2,Total,105.0,20.0,0.048,(6.0/125.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.744,0.42388678,0.92,0.0,1.0,0.8,1.0
1,auc,0.92401963,0.09907541,0.9044118,NaN,1.0,0.7916667,1.0
2,aucpr,0.2477451,0.44168508,0.9076471,NaN,0.0,0.083333336,0.0
3,err,0.09915577,0.10464366,0.08,0.21577886,0.0,0.2,0.0
4,err_count,1.75,2.362908,2.0,NaN,0.0,5.0,0.0
5,f0point5,0.647348,0.4736461,0.97402596,0.06271397,1.0,0.2,1.0
6,f1,0.8058036,0.34797573,0.9375,NaN,1.0,0.2857143,1.0
7,f2,-Infinity,NaN,0.90361446,-Infinity,1.0,0.5,1.0
8,lift_top_group,10.344203,13.390413,1.4705882,0.25042757,25.0,0.0,25.0
9,logloss,0.3012909,0.42254585,1.042946,None,0.15949012,0.21786559,0.08615286



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-01-25 10:11:20,5.037 sec,0.0,0.500000,0.693147,0.500000,0.000000,1.000000,0.840
1,,2020-01-25 10:11:20,5.043 sec,5.0,0.446181,0.590653,0.947857,0.501375,5.468750,0.080
2,,2020-01-25 10:11:20,5.048 sec,10.0,0.403737,0.515529,0.967381,0.506464,5.468750,0.072
3,,2020-01-25 10:11:20,5.053 sec,15.0,0.369649,0.457497,0.969048,0.598876,5.208333,0.056
4,,2020-01-25 10:11:20,5.058 sec,20.0,0.341196,0.409945,0.970238,0.611864,5.208333,0.064
5,,2020-01-25 10:11:20,5.064 sec,25.0,0.319747,0.374112,0.976190,0.680937,6.250000,0.056
6,,2020-01-25 10:11:20,5.070 sec,30.0,0.302877,0.345257,0.976429,0.738695,6.250000,0.056
7,,2020-01-25 10:11:20,5.075 sec,35.0,0.288508,0.320155,0.976429,0.738695,6.250000,0.056
8,,2020-01-25 10:11:20,5.080 sec,40.0,0.277181,0.300275,0.975476,0.737763,6.250000,0.056
9,,2020-01-25 10:11:21,5.087 sec,45.0,0.268326,0.284012,0.975952,0.739702,6.250000,0.056



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,winner_dga,279.737244,1.000000,0.548616
1,winner_pga,82.644966,0.295438,0.162082
2,Nominations,40.100796,0.143352,0.078645
3,year,33.642143,0.120263,0.065978
4,winner_sag,28.317915,0.101230,0.055537
5,winner_bafta,23.386477,0.083602,0.045865
6,nom_sag,17.145683,0.061292,0.033626
7,nom_bafta,2.832297,0.010125,0.005555
8,winner_gg_drama,0.841651,0.003009,0.001651
9,nom_pga,0.735163,0.002628,0.001442


In [10]:
# Re-split data on min_year to 2019
train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] >= min_year))]
test = full_table.loc[(full_table['year'] == 2019)]

# Import a binary outcome train/test set into H2O
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [11]:
top_model.train(x=x, y=y, training_frame=train)

preds = top_model.predict(test)

preds

xgboost Model Build progress: |███████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.994574,0.00542647
0,0.896084,0.103916
0,0.995754,0.00424649
0,0.964185,0.0358153
0,0.994933,0.00506749
0,0.987654,0.0123458
1,0.231437,0.768563
0,0.948777,0.0512227
0,0.6251,0.3749


In [23]:
# Feature Importance
#print(top_model.varimp)

## Predict the winner

In [12]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [13]:
test_pd[['film','probA']].sort_values('probA', ascending = False)

,film,probA
6,1917 (2019 film),0.768563
8,Parasite (2019 film),0.374900
1,The Irishman,0.103916
7,Once Upon a Time in Hollywood,0.051223
3,Joker (2019 film),0.035815
5,Marriage Story,0.012346
0,Ford v Ferrari,0.005426
4,Little Women (2019 film),0.005067
2,Jojo Rabbit,0.004246


In [14]:
test_pd.T #The DGA's (1/26) and BAFTA's (2/2) Haven't come out yet

,0,1,2,3,4,5,6,7,8
Unnamed: 0,554,555,556,557,558,559,560,561,562
year,2019,2019,2019,2019,2019,2019,2019,2019,2019
film,Ford v Ferrari,The Irishman,Jojo Rabbit,Joker (2019 film),Little Women (2019 film),Marriage Story,1917 (2019 film),Once Upon a Time in Hollywood,Parasite (2019 film)
wiki,/wiki/Ford_v_Ferrari,/wiki/The_Irishman,/wiki/Jojo_Rabbit,/wiki/Joker_(2019_film),/wiki/Little_Women_(2019_film),/wiki/Marriage_Story,/wiki/1917_(2019_film),/wiki/Once_Upon_a_Time_in_Hollywood,/wiki/Parasite_(2019_film)
winner,False,False,False,False,False,False,False,False,False
Nominations,4,10,6,11,6,6,10,10,6
Oscar_win,0,0,0,0,0,0,0,0,0
nom_gg_drama,0,1,0,1,0,1,1,0,0
winner_gg_drama,0,0,0,0,0,0,1,0,0
nom_gg_comedy,0,0,1,0,0,0,0,1,0
